# H2O GBM Tuning Tutorial for Python

### Arno Candel, PhD, Chief Architect, H2O.ai
### Ported to Python by Navdeep Gill, M.S., Hacker/Data Scientist, H2O.ai

In this tutorial, we show how to build a well-tuned H2O GBM model for a supervised classification task. We specifically don't focus on feature engineering and use a small dataset to allow you to reproduce these results in a few minutes on a laptop. This script can be directly transferred to datasets that are hundreds of GBs large and H2O clusters with dozens of compute nodes.

You can download the source [from H2O's github repository](https://github.com/h2oai/h2o-3/blob/master/h2o-docs/src/product/tutorials/gbm/gbmTuning.ipynb).

Original [R Markdown version here](https://github.com/h2oai/h2o-3/blob/master/h2o-docs/src/product/tutorials/gbm/gbmTuning.Rmd).

## Installation of the H2O Python Package
Either download H2O from [H2O.ai's website](http://h2o.ai/download) or install the latest version of H2O into Python with the following set of commands:

Install dependencies from command line (prepending with `sudo` if needed):

```
[sudo] pip install -U requests
[sudo] pip install -U tabulate
[sudo] pip install -U future
[sudo] pip install -U six
```

The following command removes the H2O module for Python.
```
[sudo] pip uninstall h2o
```

Next, use pip to install this version of the H2O Python module.
```
[sudo] pip install http://h2o-release.s3.amazonaws.com/h2o/rel-turchin/8/Python/h2o-3.8.2.8-py2.py3-none-any.whl
```

## Launch an H2O cluster on localhost

In [1]:
import h2o
import numpy as np
import math
h2o.init(nthreads=-1, strict_version_check=True)
## optional: connect to a running H2O cluster
#h2o.init(ip="mycluster", port=55555)



No instance found at ip and port: localhost:54321. Trying to start local jar...


JVM stdout: /var/folders/fm/9q0qk_xs0wd07zr2v6d5mfv40000gn/T/tmpPEJCyq/h2o_arno_started_from_python.out
JVM stderr: /var/folders/fm/9q0qk_xs0wd07zr2v6d5mfv40000gn/T/tmpU_XS5d/h2o_arno_started_from_python.err
Using ice_root: /var/folders/fm/9q0qk_xs0wd07zr2v6d5mfv40000gn/T/tmp59BuIH


Java Version: java version "1.8.0_60"
Java(TM) SE Runtime Environment (build 1.8.0_60-b27)
Java HotSpot(TM) 64-Bit Server VM (build 25.60-b23, mixed mode)


Starting H2O JVM and connecting: ............. Connection successful!


/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:98: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:672: DeprecationWarning: PlainTextFormat

H2O cluster uptime:,1 seconds 472 milliseconds
H2O cluster version:,3.8.2.8
H2O cluster name:,H2O_started_from_python_arno_txj757
H2O cluster total nodes:,1
H2O cluster total free memory:,3.56 GB
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321
H2O Connection proxy:,None


## Import the data into H2O 
Everything is scalable and distributed from now on. All processing is done on the fully multi-threaded and distributed H2O Java-based backend and can be scaled to large datasets on large compute clusters.
Here, we use a small public dataset ([Titanic](https://stat.ethz.ch/R-manual/R-devel/library/datasets/html/Titanic.html)), but you can use datasets that are hundreds of GBs large.

In [2]:
## 'path' can point to a local file, hdfs, s3, nfs, Hive, directories, etc.
df = h2o.import_file(path = "http://s3.amazonaws.com/h2o-public-test-data/smalldata/gbm_test/titanic.csv")
print df.dim
print df.head
print df.tail
print df.describe

## pick a response for the supervised problem
response = "survived"

## the response variable is an integer, we will turn it into a categorical/factor for binary classification
df[response] = df[response].asfactor()           

## use all other columns (except for the name & the response column ("survived")) as predictors
predictors = df.columns
del predictors[1:3]
print predictors


Parse Progress: [##################################################] 100%
[1309, 14]


pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
1,1,Allen Miss. Elisabeth Walton,female,29,0,0,24160,211.338,B5,S,2,nan,St Louis MO
1,1,Allison Master. Hudson Trevor,male,0.9167,1,2,113781,151.55,C22 C26,S,11,nan,Montreal PQ / Chesterville ON
1,0,Allison Miss. Helen Loraine,female,2,1,2,113781,151.55,C22 C26,S,nan,nan,Montreal PQ / Chesterville ON
1,0,Allison Mr. Hudson Joshua Creighton,male,30,1,2,113781,151.55,C22 C26,S,nan,135,Montreal PQ / Chesterville ON
1,0,Allison Mrs. Hudson J C (Bessie Waldo Daniels),female,25,1,2,113781,151.55,C22 C26,S,nan,nan,Montreal PQ / Chesterville ON
1,1,Anderson Mr. Harry,male,48,0,0,19952,26.55,E12,S,3,nan,New York NY
1,1,Andrews Miss. Kornelia Theodosia,female,63,1,0,13502,77.9583,D7,S,10,nan,Hudson NY
1,0,Andrews Mr. Thomas Jr,male,39,0,0,112050,0,A36,S,nan,nan,Belfast NI
1,1,Appleton Mrs. Edward Dale (Charlotte Lamson),female,53,2,0,11769,51.4792,C101,S,nan,nan,Bayside Queens NY
1,0,Artagaveytia Mr. Ramon,male,71,0,0,nan,49.5042,,C,nan,22,Montevideo Uruguay


<bound method H2OFrame.head of >


pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
1,1,Allen Miss. Elisabeth Walton,female,29,0,0,24160,211.338,B5,S,2,nan,St Louis MO
1,1,Allison Master. Hudson Trevor,male,0.9167,1,2,113781,151.55,C22 C26,S,11,nan,Montreal PQ / Chesterville ON
1,0,Allison Miss. Helen Loraine,female,2,1,2,113781,151.55,C22 C26,S,nan,nan,Montreal PQ / Chesterville ON
1,0,Allison Mr. Hudson Joshua Creighton,male,30,1,2,113781,151.55,C22 C26,S,nan,135,Montreal PQ / Chesterville ON
1,0,Allison Mrs. Hudson J C (Bessie Waldo Daniels),female,25,1,2,113781,151.55,C22 C26,S,nan,nan,Montreal PQ / Chesterville ON
1,1,Anderson Mr. Harry,male,48,0,0,19952,26.55,E12,S,3,nan,New York NY
1,1,Andrews Miss. Kornelia Theodosia,female,63,1,0,13502,77.9583,D7,S,10,nan,Hudson NY
1,0,Andrews Mr. Thomas Jr,male,39,0,0,112050,0,A36,S,nan,nan,Belfast NI
1,1,Appleton Mrs. Edward Dale (Charlotte Lamson),female,53,2,0,11769,51.4792,C101,S,nan,nan,Bayside Queens NY
1,0,Artagaveytia Mr. Ramon,male,71,0,0,nan,49.5042,,C,nan,22,Montevideo Uruguay


<bound method H2OFrame.tail of >


pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
1,1,Allen Miss. Elisabeth Walton,female,29,0,0,24160,211.338,B5,S,2,nan,St Louis MO
1,1,Allison Master. Hudson Trevor,male,0.9167,1,2,113781,151.55,C22 C26,S,11,nan,Montreal PQ / Chesterville ON
1,0,Allison Miss. Helen Loraine,female,2,1,2,113781,151.55,C22 C26,S,nan,nan,Montreal PQ / Chesterville ON
1,0,Allison Mr. Hudson Joshua Creighton,male,30,1,2,113781,151.55,C22 C26,S,nan,135,Montreal PQ / Chesterville ON
1,0,Allison Mrs. Hudson J C (Bessie Waldo Daniels),female,25,1,2,113781,151.55,C22 C26,S,nan,nan,Montreal PQ / Chesterville ON
1,1,Anderson Mr. Harry,male,48,0,0,19952,26.55,E12,S,3,nan,New York NY
1,1,Andrews Miss. Kornelia Theodosia,female,63,1,0,13502,77.9583,D7,S,10,nan,Hudson NY
1,0,Andrews Mr. Thomas Jr,male,39,0,0,112050,0,A36,S,nan,nan,Belfast NI
1,1,Appleton Mrs. Edward Dale (Charlotte Lamson),female,53,2,0,11769,51.4792,C101,S,nan,nan,Bayside Queens NY
1,0,Artagaveytia Mr. Ramon,male,71,0,0,nan,49.5042,,C,nan,22,Montevideo Uruguay


<bound method H2OFrame.describe of >
[u'pclass', u'sex', u'age', u'sibsp', u'parch', u'ticket', u'fare', u'cabin', u'embarked', u'boat', u'body', u'home.dest']


From now on, everything is generic and directly applies to most datasets. We assume that all feature engineering is done at this stage and focus on model tuning. For multi-class problems, you can use `h2o.logloss()` or `h2o.confusion_matrix()` instead of `h2o.auc()` and for regression problems, you can use `h2o.mean_residual_deviance()` or `h2o.mse()`.

## Split the data for Machine Learning
We split the data into three pieces: 60% for training, 20% for validation, 20% for final testing. 
Here, we use random splitting, but this assumes i.i.d. data. If this is not the case (e.g., when events span across multiple rows or data has a time structure), you'll have to sample your data non-randomly.

In [3]:
train, valid, test = df.split_frame(
    ratios=[0.6,0.2], 
    seed=1234, 
    destination_frames=['train.hex','valid.hex','test.hex']
)

## Establish baseline performance
As the first step, we'll build some default models to see what accuracy we can expect. Let's use the [AUC metric](http://mlwiki.org/index.php/ROC_Analysis) for this demo, but you can use `h2o.logloss()` and `stopping_metric="logloss"` as well. It ranges from 0.5 for random models to 1 for perfect models.


The first model is a default GBM, trained on the 60% training split

In [4]:
#We only provide the required parameters, everything else is default
gbm = h2o.H2OGradientBoostingEstimator(distribution='bernoulli')
gbm.train(x=predictors, y=response, training_frame=train)

## Show a detailed model summary
print gbm


gbm Model Build Progress: [##################################################] 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Method
Model Key:  GBM_model_python_1465581184222_1

Model Summary: 


,number_of_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,50.0,27314.0,5.0,5.0,5.0,10.0,21.0,15.58




ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.0347141508428
R^2: 0.853514801928
LogLoss: 0.135724386711
Mean Per-Class Error: 0.0403838284355
AUC: 0.990369609999
Gini: 0.980739219997

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.394257736927: 


,0,1,Error,Rate
0,461.0,18.0,0.0376,(18.0/479.0)
1,13.0,288.0,0.0432,(13.0/301.0)
Total,474.0,306.0,0.0397,(31.0/780.0)



Maximum Metrics: Maximum metrics at their respective thresholds



...

,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0102564,0.9792763,2.5913621,2.5913621,1.0,1.0,0.0265781,0.0265781,159.1362126,159.1362126
,2,0.0205128,0.9772307,2.5913621,2.5913621,1.0,1.0,0.0265781,0.0531561,159.1362126,159.1362126
,3,0.0307692,0.9749052,2.5913621,2.5913621,1.0,1.0,0.0265781,0.0797342,159.1362126,159.1362126
,4,0.0410256,0.9735284,2.5913621,2.5913621,1.0,1.0,0.0265781,0.1063123,159.1362126,159.1362126
,5,0.0525641,0.9727573,2.5913621,2.5913621,1.0,1.0,0.0299003,0.1362126,159.1362126,159.1362126
,6,0.1,0.9705040,2.5913621,2.5913621,1.0,1.0,0.1229236,0.2591362,159.1362126,159.1362126
,7,0.15,0.9690187,2.5913621,2.5913621,1.0,1.0,0.1295681,0.3887043,159.1362126,159.1362126
,8,0.2,0.9666881,2.5913621,2.5913621,1.0,1.0,0.1295681,0.5182724,159.1362126,159.1362126
,9,0.3,0.9286900,2.5913621,2.5913621,1.0,1.0,0.2591362,0.7774086,159.1362126,159.1362126
,10,0.4,0.3392288,1.8272425,2.4003322,0.7051282,0.9262821,0.1827243,0.9601329,82.7242525,140.0332226




Scoring History: 


,timestamp,duration,number_of_trees,training_MSE,training_logloss,training_AUC,training_lift,training_classification_error
,2016-06-10 10:53:14,0.013 sec,0.0,0.2369806,0.6668775,0.5,1.0,0.6141026
,2016-06-10 10:53:14,0.134 sec,1.0,0.2022753,0.5942842,0.9685634,2.5913621,0.0782051
,2016-06-10 10:53:14,0.172 sec,2.0,0.1744836,0.5361788,0.9760888,2.5913621,0.0692308
,2016-06-10 10:53:14,0.201 sec,3.0,0.1520389,0.4883150,0.9760888,2.5913621,0.0692308
,2016-06-10 10:53:14,0.231 sec,4.0,0.1335759,0.4476644,0.9767338,2.5913621,0.0692308
---,---,---,---,---,---,---,---,---
,2016-06-10 10:53:15,1.047 sec,46.0,0.0351867,0.1373187,0.9900159,2.5913621,0.0410256
,2016-06-10 10:53:15,1.067 sec,47.0,0.0350182,0.1368714,0.9901338,2.5913621,0.0410256
,2016-06-10 10:53:15,1.085 sec,48.0,0.0349213,0.1365776,0.9902794,2.5913621,0.0410256
,2016-06-10 10:53:15,1.101 sec,49.0,0.0347513,0.1361386,0.9903765,2.5913621,0.0397436



See the whole table with table.as_data_frame()

Variable Importances: 


variable,relative_importance,scaled_importance,percentage
boat,608.0037231,1.0,0.5470799
home.dest,379.0013123,0.6233536,0.3410242
cabin,35.3481979,0.0581381,0.0318062
sex,28.0973225,0.0462124,0.0252819
ticket,21.2186050,0.0348988,0.0190924
embarked,14.9458570,0.0245819,0.0134482
fare,11.1052256,0.0182651,0.0099924
age,5.4856591,0.0090224,0.0049360
parch,3.7122171,0.0061056,0.0033402
sibsp,1.8749361,0.0030838,0.0016871


The AUC is over 94%, so this model is highly predictive!

In [5]:
## Get the AUC on the validation set
perf = gbm.model_performance(valid)
print perf.auc()

0.943195266272


The second model is another default GBM, but trained on 80% of the data (here, we combine the training and validation splits to get more training data), and cross-validated using 4 folds.
Note that cross-validation takes longer and is not usually done for really large datasets.

In [6]:
## rbind() makes a copy here, so it's better to use split_frame with `ratios = c(0.8)` instead above
cv_gbm = h2o.H2OGradientBoostingEstimator(distribution='bernoulli',nfolds = 4, seed = 0xDECAF)
cv_gbm.train(x = predictors, y = response, training_frame = train.rbind(valid))


gbm Model Build Progress: [##################################################] 100%


We see that the cross-validated performance is similar to the validation set performance:

In [7]:
## Show a detailed summary of the cross validation metrics
## This gives you an idea of the variance between the folds
cv_summary = cv_gbm.cross_validation_metrics_summary().as_data_frame()
#print(cv_summary) ## Full summary of all metrics
#print(cv_summary.iloc[4]) ## get the row with just the AUCs

## Get the cross-validated AUC by scoring the combined holdout predictions.
## (Instead of taking the average of the metrics across the folds)
perf_cv = cv_gbm.model_performance(xval=True)
print perf_cv.auc()

0.940343155142


Next, we train a GBM with "I feel lucky" parameters.
We'll use early stopping to automatically tune the number of trees using the validation AUC. 
We'll use a lower learning rate (lower is always better, just takes more trees to converge).
We'll also use stochastic sampling of rows and columns to (hopefully) improve generalization.

In [8]:
gbm_lucky = h2o.H2OGradientBoostingEstimator(
  ## more trees is better if the learning rate is small enough 
  ## here, use "more than enough" trees - we have early stopping
  ntrees = 10000,                                                            

  ## smaller learning rate is better (this is a good value for most datasets, but see below for annealing)
  learn_rate = 0.01,                                                         

  ## early stopping once the validation AUC doesn't improve by at least 0.01% for 5 consecutive scoring events
  stopping_rounds = 5, stopping_tolerance = 1e-4, stopping_metric = "AUC", 

  ## sample 80% of rows per tree
  sample_rate = 0.8,                                                       

  ## sample 80% of columns per split
  col_sample_rate = 0.8,                                                   

  ## fix a random number generator seed for reproducibility
  seed = 1234,                                                             

  ## score every 10 trees to make early stopping reproducible (it depends on the scoring interval)
  score_tree_interval = 10)

gbm_lucky.train(x=predictors, y=response, training_frame=train, validation_frame=valid)


gbm Model Build Progress: [##################################################] 100%


This model doesn't seem to be much better than the previous models:

In [9]:
perf_lucky = gbm_lucky.model_performance(valid)
print perf_lucky.auc()

0.93933502395


For this small dataset, dropping 20% of observations per tree seems too aggressive in terms of adding regularization. For larger datasets, this is usually not a bad idea. But we'll let this parameter tune freshly below, so no worries.

## Hyper-Parameter Search

Next, we'll do real hyper-parameter optimization to see if we can beat the best AUC so far (around 94%).

The key here is to start tuning some key parameters first (i.e., those that we expect to have the biggest impact on the results). From experience with gradient boosted trees across many datasets, we can state the following "rules":

1. Build as many trees (`ntrees`) as it takes until the validation set error starts increasing.
2. A lower learning rate (`learn_rate`) is generally better, but will require more trees. Using `learn_rate=0.02 `and `learn_rate_annealing=0.995` (reduction of learning rate with each additional tree) can help speed up convergence without sacrificing accuracy too much, and is great to hyper-parameter searches. For faster scans, use values of 0.05 and 0.99 instead.
3. The optimum maximum allowed depth for the trees (`max_depth`) is data dependent, deeper trees take longer to train, especially at depths greater than 10.
4. Row and column sampling (`sample_rate` and `col_sample_rate`) can improve generalization and lead to lower validation and test set errors. Good general values for large datasets are around 0.7 to 0.8 (sampling 70-80 percent of the data) for both parameters. Column sampling per tree (`col_sample_rate_per_tree`) can also be tuned. Note that it is multiplicative with `col_sample_rate`, so setting both parameters to 0.8 results in 64% of columns being considered at any given node to split.
5. For highly imbalanced classification datasets (e.g., fewer buyers than non-buyers), stratified row sampling based on response class membership can help improve predictive accuracy.  It is configured with `sample_rate_per_class` (array of ratios, one per response class in lexicographic order).
6. Most other options only have a small impact on the model performance, but are worth tuning with a Random hyper-parameter search nonetheless, if highest performance is critical.

First we want to know what value of `max_depth` to use because it has a big impact on the model training time and optimal values depend strongly on the dataset.
We'll do a quick Cartesian grid search to get a rough idea of good candidate `max_depth` values. Each model in the grid search will use early stopping to tune the number of trees using the validation set AUC, as before.
We'll use learning rate annealing to speed up convergence without sacrificing too much accuracy.

In [10]:
## Depth 10 is usually plenty of depth for most datasets, but you never know
hyper_params = {'max_depth' : range(1,30,2)}
#hyper_params = {max_depth = [4,6,8,12,16,20]} ##faster for larger datasets

#Build initial GBM Model
gbm_grid = h2o.H2OGradientBoostingEstimator(distribution='bernoulli',
        ## more trees is better if the learning rate is small enough 
        ## here, use "more than enough" trees - we have early stopping
        ntrees=10000,
        ## smaller learning rate is better
        ## since we have learning_rate_annealing, we can afford to start with a 
        #bigger learning rate
        learn_rate=0.05,
        ## learning rate annealing: learning_rate shrinks by 1% after every tree 
        ## (use 1.00 to disable, but then lower the learning_rate)
        learn_rate_annealing = 0.99,
        ## sample 80% of rows per tree
        sample_rate = 0.8,
        ## sample 80% of columns per split
        col_sample_rate = 0.8,
        ## fix a random number generator seed for reproducibility
        seed = 1234,
        ## score every 10 trees to make early stopping reproducible 
        #(it depends on the scoring interval)
        score_tree_interval = 10, 
        ## early stopping once the validation AUC doesn't improve by at least 0.01% for 
        #5 consecutive scoring events
        stopping_rounds = 5,
        stopping_metric = "AUC",
        stopping_tolerance = 1e-4)

#Build grid search with previously made GBM and hyper parameters
grid = h2o.H2OGridSearch(gbm_grid,hyper_params,
                         grid_id = 'depth_grid',
                         search_criteria = {'strategy': "Cartesian"})


#Train grid search
grid.train(x=predictors, 
           y=response,
           training_frame = train,
           validation_frame = valid)


gbm Grid Build Progress: [##################################################] 100%


In [12]:
## by default, display the grid search results sorted by increasing logloss (since this is a classification task)
print grid

      max_depth            model_ids   logloss
0            25  depth_grid_model_12  0.215749
1            23  depth_grid_model_11  0.216639
2            29  depth_grid_model_14  0.218503
3            13   depth_grid_model_6  0.218825
4            19   depth_grid_model_9  0.218849
5            27  depth_grid_model_13  0.219126
6            21  depth_grid_model_10  0.219794
7            17   depth_grid_model_8  0.220059
8            11   depth_grid_model_5  0.221247
9             9   depth_grid_model_4  0.222040
10           15   depth_grid_model_7  0.222911
11            7   depth_grid_model_3  0.225722
12            5   depth_grid_model_2  0.232193
13            3   depth_grid_model_1  0.264067
14            1   depth_grid_model_0  0.324154



In [13]:
## sort the grid models by decreasing AUC
sorted_grid = grid.get_grid(sort_by='auc',decreasing=True)
print(sorted_grid)

      max_depth            model_ids       auc
0            27  depth_grid_model_13  0.956579
1            25  depth_grid_model_12  0.956354
2            29  depth_grid_model_14  0.956241
3            21  depth_grid_model_10  0.954663
4            19   depth_grid_model_9  0.954494
5            13   depth_grid_model_6  0.954382
6            23  depth_grid_model_11  0.954043
7            11   depth_grid_model_5  0.952184
8            15   depth_grid_model_7  0.951789
9            17   depth_grid_model_8  0.951507
10            9   depth_grid_model_4  0.950437
11            7   depth_grid_model_3  0.946943
12            5   depth_grid_model_2  0.939307
13            3   depth_grid_model_1  0.932713
14            1   depth_grid_model_0  0.929022



It appears that `max_depth` values of 19 to 29 are best suited for this dataset, which is unusally deep!

In [14]:
max_depths = sorted_grid.sorted_metric_table()['max_depth'][0:5]
new_max = max(max_depths, key=int)
new_min = min(max_depths, key=int)

print "MaxDepth", new_max
print "MinDepth", new_min

MaxDepth 29
MinDepth 19


Now that we know a good range for max_depth, we can tune all other parameters in more detail. Since we don't know what combinations of hyper-parameters will result in the best model, we'll use random hyper-parameter search to "let the machine get luckier than a best guess of any human".

In [15]:
# create hyperameter and search criteria lists (ranges are inclusive..exclusive))
hyper_params_tune = {'max_depth' : list(range(new_min,new_max+1,1)),
                'sample_rate': [x/100. for x in range(20,101)],
                'col_sample_rate' : [x/100. for x in range(20,101)],
                'col_sample_rate_per_tree': [x/100. for x in range(20,101)],
                'col_sample_rate_change_per_level': [x/100. for x in range(90,111)],
                'min_rows': [2**x for x in range(0,int(math.log(train.nrow,2)-1)+1)],
                'nbins': [2**x for x in range(4,11)],
                'nbins_cats': [2**x for x in range(4,13)],
                'min_split_improvement': [0,1e-8,1e-6,1e-4],
                'histogram_type': ["UniformAdaptive","QuantilesGlobal","RoundRobin"]}
search_criteria_tune = {'strategy': "RandomDiscrete",
                   'max_runtime_secs': 3600,  ## limit the runtime to 60 minutes
                   'max_models': 100,  ## build no more than 100 models
                   'seed' : 1234,
                   'stopping_rounds' : 5,
                   'stopping_metric' : "AUC",
                   'stopping_tolerance': 1e-3
                   }

In [16]:
gbm_final_grid = h2o.H2OGradientBoostingEstimator(distribution='bernoulli',
                    ## more trees is better if the learning rate is small enough 
                    ## here, use "more than enough" trees - we have early stopping
                    ntrees=10000,
                    ## smaller learning rate is better
                    ## since we have learning_rate_annealing, we can afford to start with a 
                    #bigger learning rate
                    learn_rate=0.05,
                    ## learning rate annealing: learning_rate shrinks by 1% after every tree 
                    ## (use 1.00 to disable, but then lower the learning_rate)
                    learn_rate_annealing = 0.99,
                    ## score every 10 trees to make early stopping reproducible 
                    #(it depends on the scoring interval)
                    score_tree_interval = 10,
                    ## fix a random number generator seed for reproducibility
                    seed = 1234,
                    ## early stopping once the validation AUC doesn't improve by at least 0.01% for 
                    #5 consecutive scoring events
                    stopping_rounds = 5,
                    stopping_metric = "AUC",
                    stopping_tolerance = 1e-4)
            
#Build grid search with previously made GBM and hyper parameters
final_grid = h2o.H2OGridSearch(gbm_final_grid, hyper_params = hyper_params_tune,
                                    grid_id = 'final_grid',
                                    search_criteria = search_criteria_tune)
#Train grid search
final_grid.train(x=predictors, 
           y=response,
           ## early stopping based on timeout (no model should take more than 1 hour - modify as needed)
           max_runtime_secs = 3600, 
           training_frame = train,
           validation_frame = valid)

print final_grid


gbm Grid Build Progress: [##################################################] 100%
       col_sample_rate  col_sample_rate_change_per_level  \
0                 0.49                              1.04   
1                 0.92                              0.93   
2                 0.47                              0.98   
3                 0.35                              1.09   
4                 0.42                              0.98   
5                 0.70                              1.02   
6                 0.60                              1.00   
7                 0.55                              1.05   
8                 0.50                              1.02   
9                 0.40                              1.01   
10                0.61                              1.04   
11                0.52                              0.97   
12                0.20                              0.96   
13                0.50                              0.94   
14              

We can see that the best models have even better validation AUCs than our previous best models, so the random grid search was successful!

In [19]:
## Sort the grid models by AUC
sorted_final_grid = final_grid.get_grid(sort_by='auc',decreasing=True)

print sorted_final_grid

       col_sample_rate  col_sample_rate_change_per_level  \
0                 0.49                              1.04   
1                 0.92                              0.93   
2                 0.35                              1.09   
3                 0.42                              0.98   
4                 0.70                              1.02   
5                 0.47                              0.98   
6                 0.50                              1.02   
7                 0.20                              0.96   
8                 0.40                              1.01   
9                 0.50                              0.94   
10                0.91                              0.96   
11                0.90                              1.10   
12                0.66                              0.96   
13                0.60                              1.00   
14                0.42                              1.08   
15                0.96                  

You can also see the results of the grid search in [Flow](http://localhost:54321/):
![alt text](./final_grid.png "Final Grid Search Results in Flow")

In [ ]:
sorted_final_grid

## Model Inspection and Final Test Set Scoring

Let's see how well the best model of the grid search (as judged by validation set AUC) does on the held out test set:

In [21]:
#Get the best model from the list (the model name listed at the top of the table)
best_model = h2o.get_model(sorted_final_grid.sorted_metric_table()['model_ids'][0])
performance_best_model = best_model.model_performance(test)

Good news. It does as well on the test set as on the validation set, so it looks like our best GBM model generalizes well to the unseen test set:

In [22]:
#Get the performance on the test model
print performance_best_model.auc()

0.971256772829


We can inspect the winning model's parameters:

In [23]:
params_list = []
for key, value in best_model.params.iteritems():
    params_list.append(str(key)+" = "+str(value['actual']))
params_list

['learn_rate = 0.05',
 'fold_column = None',
 'col_sample_rate_per_tree = 0.94',
 'learn_rate_annealing = 0.99',
 'score_tree_interval = 10',
 'sample_rate_per_class = None',
 'seed = 1234',
 'keep_cross_validation_predictions = False',
 "model_id = {u'URL': u'/4/Models/final_grid_model_68', u'type': u'Key<Model>', u'name': u'final_grid_model_68', u'__meta': {u'schema_name': u'ModelKeyV3', u'schema_version': 3, u'schema_type': u'Key<Model>'}}",
 'nfolds = 0',
 'max_abs_leafnode_pred = 1.79769313486e+308',
 'offset_column = None',
 'quantile_alpha = 0.5',
 'stopping_tolerance = 0.0001',
 'fold_assignment = AUTO',
 "training_frame = {u'URL': u'/4/Frames/train.hex', u'type': u'Key<Frame>', u'name': u'train.hex', u'__meta': {u'schema_name': u'FrameKeyV3', u'schema_version': 3, u'schema_type': u'Key<Frame>'}}",
 'max_runtime_secs = 3438.745',
 'checkpoint = None',
 'balance_classes = False',
 'r2_stopping = 0.999999',
 "validation_frame = {u'URL': u'/4/Frames/valid.hex', u'type': u'Key<Fram

Now we can confirm that these parameters are generally sound, by building a GBM model on the whole dataset (instead of the 60%) and using internal 5-fold cross-validation (re-using all other parameters including the seed):

In [25]:
gbm = h2o.get_model(sorted_final_grid.sorted_metric_table()['model_ids'][0])
#get the parameters from the Random grid search model and modify them slightly
params = gbm.params
new_params = {"nfolds":5, "model_id":None}
for key in new_params.keys():
    params[key]['actual'] = new_params[key] 
gbm_best = h2o.H2OGradientBoostingEstimator()
for key in params.keys():
    if key in dir(gbm_best) and getattr(gbm_best,key) != params[key]['actual']:
        setattr(gbm_best,key,params[key]['actual']) 

In [26]:
gbm_best.train(x=predictors, y=response, training_frame=df)


gbm Model Build Progress: [##################################################] 100%


In [27]:
print gbm_best.cross_validation_metrics_summary()


Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
F0point5,0.9082877,0.0174698,0.9448819,0.8739837,0.8935743,0.9034908,0.9255079
F1,0.8978795,0.0085111,0.9099526,0.8820513,0.8989899,0.9119171,0.8864865
F2,0.8886758,0.0168452,0.8775137,0.8902692,0.9044715,0.9205021,0.8506224
accuracy,0.9236877,0.0046046,0.9288390,0.9151291,0.9224806,0.9330708,0.9189189
auc,0.9606385,0.0066715,0.9664746,0.9453869,0.959375,0.9737173,0.9582387
err,0.0763123,0.0046046,0.0711611,0.0848708,0.0775194,0.0669291,0.0810811
err_count,20.0,1.4142135,19.0,23.0,20.0,17.0,21.0
lift_top_group,2.6258688,0.0998947,2.3839285,2.8229167,2.632653,2.6736841,2.6161616
logloss,0.2343099,0.0190066,0.2362470,0.2616568,0.2454384,0.1831158,0.2450912
max_per_class_error,0.1168524,0.0251726,0.1428571,0.1041667,0.0918367,0.0736842,0.1717172


Ouch! So it looks like we overfit quite a bit on the validation set as the mean AUC on the 5 folds is "only" 96.06% +/- 0.67%. So we cannot always expect AUCs of 97% with these parameters on this dataset. So to get a better estimate of model performance, the Random hyper-parameter search could have used `nfolds = 5` (or 10, or similar) in combination with 80% of the data for training (i.e., not holding out a validation set, but only the final test set). However, this would take more time, as `nfolds+1` models will be built for every set of parameters.

Instead, to save time, lets just scan through the top 5 models and cross validate their parameters with `nfolds = 5` on the entire dataset:

In [28]:
for i in range(5): 
    gbm = h2o.get_model(sorted_final_grid.sorted_metric_table()['model_ids'][i])
    #get the parameters from the Random grid search model and modify them slightly
    params = gbm.params
    new_params = {"nfolds":5, "model_id":None}
    for key in new_params.keys():
        params[key]['actual'] = new_params[key]
    new_model = h2o.H2OGradientBoostingEstimator()
    for key in params.keys():
        if key in dir(new_model) and getattr(new_model,key) != params[key]['actual']:
            setattr(new_model,key,params[key]['actual'])
    new_model.train(x = predictors, y = response, training_frame = df)  
    cv_summary = new_model.cross_validation_metrics_summary().as_data_frame()
    print(gbm.model_id)
    print(cv_summary.iloc[4]) ## AUC


gbm Model Build Progress: [##################################################] 100%
final_grid_model_68
                     auc
mean            0.960638
sd            0.00667145
cv_1_valid      0.966475
cv_2_valid      0.945387
cv_3_valid      0.959375
cv_4_valid      0.973717
cv_5_valid      0.958239
Name: 4, dtype: object

gbm Model Build Progress: [##################################################] 100%
final_grid_model_96
                     auc
mean            0.964915
sd            0.00522182
cv_1_valid      0.963191
cv_2_valid      0.959702
cv_3_valid      0.974298
cv_4_valid      0.972393
cv_5_valid      0.954987
Name: 4, dtype: object

gbm Model Build Progress: [##################################################] 100%
final_grid_model_38
                    auc
mean           0.963851
sd            0.0046032
cv_1_valid     0.961348
cv_2_valid     0.957351
cv_3_valid     0.971301
cv_4_valid      0.97193
cv_5_valid     0.957323
Name: 4, dtype: object

gbm Model Build Progres

The avid reader might have noticed that we just implicitly did further parameter tuning using the "final" test set (which is part of the entire dataset `df`), which is not good practice - one is not supposed to use the "final" test set more than once. Hence, we're not going to pick a different "best" model, but we're just learning about the variance in AUCs. It turns out, for this tiny dataset, that the variance is rather large, which is not surprising.

Keeping the same "best" model, we can make test set predictions as follows:

In [29]:
preds = best_model.predict(test)
preds.head()


gbm prediction Progress: [##################################################] 100%


predict,p0,p1
0,0.980559,0.0194406
0,0.980512,0.019488
0,0.81431,0.18569
1,0.0212124,0.978788
1,0.025281,0.974719
0,0.92056,0.0794398
1,0.0322029,0.967797
1,0.0279367,0.972063
1,0.0304842,0.969516
0,0.906846,0.0931544


Note that the label (survived or not) is predicted as well (in the first predict column), and it uses the threshold with the highest F1 score (here: 0.528098) to make labels from the probabilities for survival (`p1`). The probability for death (`p0`) is given for convenience, as it is just `1-p1`.

In [30]:
best_model.model_performance(valid)


ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.0479062825655
R^2: 0.797316874055
LogLoss: 0.178131262416
Mean Per-Class Error: 0.0676810369118
AUC: 0.974049027895
Gini: 0.94809805579

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.528097833353: 


,0,1,Error,Rate
0,165.0,4.0,0.0237,(4.0/169.0)
1,12.0,93.0,0.1143,(12.0/105.0)
Total,177.0,97.0,0.0584,(16.0/274.0)



Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.5280978,0.9207921,96.0
max f2,0.1708529,0.9269663,113.0
max f0point5,0.7679312,0.9594883,90.0
max accuracy,0.7679312,0.9416058,90.0
max precision,0.9794488,1.0,0.0
max recall,0.0194250,1.0,206.0
max specificity,0.9794488,1.0,0.0
max absolute_MCC,0.7679312,0.8786916,90.0
max min_per_class_accuracy,0.2044670,0.9289941,109.0
max mean_per_class_accuracy,0.2524734,0.9323190,106.0



Gains/Lift Table: Avg response rate: 38.32 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0109489,0.9789164,2.6095238,2.6095238,1.0,1.0,0.0285714,0.0285714,160.9523810,160.9523810
,2,0.0218978,0.9786176,2.6095238,2.6095238,1.0,1.0,0.0285714,0.0571429,160.9523810,160.9523810
,3,0.0328467,0.9784185,2.6095238,2.6095238,1.0,1.0,0.0285714,0.0857143,160.9523810,160.9523810
,4,0.0401460,0.9780194,2.6095238,2.6095238,1.0,1.0,0.0190476,0.1047619,160.9523810,160.9523810
,5,0.0510949,0.9778627,2.6095238,2.6095238,1.0,1.0,0.0285714,0.1333333,160.9523810,160.9523810
,6,0.1021898,0.9751409,2.6095238,2.6095238,1.0,1.0,0.1333333,0.2666667,160.9523810,160.9523810
,7,0.1496350,0.9701773,2.6095238,2.6095238,1.0,1.0,0.1238095,0.3904762,160.9523810,160.9523810
,8,0.2007299,0.9675483,2.6095238,2.6095238,1.0,1.0,0.1333333,0.5238095,160.9523810,160.9523810
,9,0.2992701,0.9100651,2.6095238,2.6095238,1.0,1.0,0.2571429,0.7809524,160.9523810,160.9523810
,10,0.4014599,0.2036354,1.4911565,2.3248485,0.5714286,0.8909091,0.1523810,0.9333333,49.1156463,132.4848485


You can also see the "best" model in more detail in [Flow](http://localhost:54321/):
![alt text](./best_gbm1.png "Best GBM from Grid Search in Flow")
![alt text](./best_gbm2.png "Best GBM from Grid Search in Flow")

The model and the predictions can be saved to file as follows:

In [31]:
h2o.save_model(best_model, "/tmp/bestModel.csv", force=True)
h2o.export_file(preds, "/tmp/bestPreds.csv", force=True)


Export File Progress: [##################################################] 100%


In [32]:
#h2o.download_pojo(best_model)

The model can also be exported as a plain old Java object (POJO) for H2O-independent (standalone/Storm/Kafka/UDF) scoring in any Java environment.

```
/*
  Licensed under the Apache License, Version 2.0
    http://www.apache.org/licenses/LICENSE-2.0.html

  AUTOGENERATED BY H2O at 2016-05-27T22:11:18.485-07:00
  3.8.2.6
  
  Standalone prediction code with sample test data for GBMModel named final_grid_model_46

  How to download, compile and execute:
      mkdir tmpdir
      cd tmpdir
      curl http://127.0.0.1:54321/3/h2o-genmodel.jar > h2o-genmodel.jar
      curl http://127.0.0.1:54321/3/Models.java/final_grid_model_46 > final_grid_model_46.java
      javac -cp h2o-genmodel.jar -J-Xmx2g -J-XX:MaxPermSize=128m final_grid_model_46.java

     (Note:  Try java argument -XX:+PrintCompilation to show runtime JIT compiler behavior.)
*/
...
class final_grid_model_46_Tree_289_class_0 {
  static final double score0(double[] data) {
    double pred =      (!Double.isNaN(data[9]) && data[9 /* boat */] != 12.0f ? 
         (data[9 /* boat */] <13.003977f ? 
             (data[9 /* boat */] <11.504464f ? 
                 (data[1 /* sex */] <0.5f ? 
                     (data[6 /* fare */] <24.978184f ? 
                        0.0027945167f : 
                         (data[11 /* home.dest */] <132.0f ? 
                            0.0027875851f : 
                             (data[11 /* home.dest */] <265.5f ? 
                                 (data[6 /* fare */] <49.690052f ? 
                                    0.002784563f : 
                                    0.0027828787f) : 
                                0.0027856496f))) : 
                    0.002798641f) : 
                0.0028186426f) : 
            0.00350058f) : 
...
```

## Ensembling Techniques

After learning above that the variance of the test set AUC of the top few models was rather large, we might be able to turn this into our advantage by using ensembling techniques. The simplest one is taking the average of the predictions (survival probabilities) of the top `k` grid search model predictions (here, we use `k=10`):

In [34]:
prob = None
k=10
for i in range(0,k): 
    gbm = h2o.get_model(sorted_final_grid.sorted_metric_table()['model_ids'][i])
    if (prob is None):
        prob = gbm.predict(test)["p1"]
    else:
        prob = prob + gbm.predict(test)["p1"]
prob = prob/k


gbm prediction Progress: [##################################################] 100%

gbm prediction Progress: [##################################################] 100%

gbm prediction Progress: [##################################################] 100%

gbm prediction Progress: [##################################################] 100%

gbm prediction Progress: [##################################################] 100%

gbm prediction Progress: [##################################################] 100%

gbm prediction Progress: [##################################################] 100%

gbm prediction Progress: [##################################################] 100%

gbm prediction Progress: [##################################################] 100%

gbm prediction Progress: [##################################################] 100%


We now have a blended probability of survival for each person on the Titanic.

In [35]:
prob.head()

p1
0.0225892
0.0161596
0.158373
0.985657
0.987922
0.179414
0.956481
0.983956
0.962046
0.0886235


We can bring those ensemble predictions to our Python session's memory space and use other Python packages.

In [36]:
from sklearn.metrics import roc_auc_score
# convert prob and test[response] h2oframes to pandas' frames and then convert them each to numpy array
np_array_prob = prob.as_data_frame().as_matrix()
np_array_test = test[response].as_data_frame().as_matrix()
probInPy = np_array_prob
labeInPy = np_array_test
# compare true scores (test[response]) to probability scores (prob)
roc_auc_score(labeInPy, probInPy)

0.97753402933791456

This simple blended ensemble test set prediction has an even higher AUC than the best single model, but we need to do more validation studies, ideally using cross-validation. We leave this as an exercise for the reader - take the parameters of the top `10` models, retrain them with `nfolds=5` on the full dataset, set `keep_holdout_predictions=TRUE` and sum up their predicted probabilities, then score that with sklearn's roc_auc_score as shown above.

For more sophisticated ensembling approaches, such as stacking via a superlearner, we refer to the [H2O Ensemble](https://github.com/h2oai/h2o-3/tree/master/h2o-r/ensemble) github page.

## Summary
We learned how to build H2O GBM models for a binary classification task on a small but realistic dataset with numerical and categorical variables, with the goal to maximize the AUC (ranges from 0.5 to 1). We first established a baseline with the default model, then carefully tuned the remaining hyper-parameters without "too much" human guess-work. We used both Cartesian and Random hyper-parameter searches to find good models. We were able to get the AUC on a holdout test set from the low 94% range with the default model to the mid 97% after tuning, and to the high 97% with some simple ensembling technique known as blending. We performed simple cross-validation variance analysis to learn that results were slightly "lucky" due to the specific train/valid/test set splits, and settled to expect mid 96% AUCs instead.

Note that this script and the findings therein are directly transferrable to large datasets on distributed clusters including Spark/Hadoop environments.

More information can be found here [http://www.h2o.ai/docs/](http://www.h2o.ai/docs/).